In [51]:
from PIL import Image
import os
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# set results_folder to the folder in the same directory with the name "results"
results_folder = '../smallresults/'
train_folder = '../smallresults/train/'
test_folder = '../smallresults/test/'
validation_folder = '../smallresults/val/'

In [52]:
# Create a model
model = Sequential(name='verydeeplearning')
# Add a convolutional layer with 32 filters, each of size 3x3
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))

# Add convolutional layers 
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))

# Output
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

In [53]:
# Compile the model
model.compile(Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
# Create a generator that will take the data and create batches of grayscale images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_folder, target_size=(200, 200), batch_size=22, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_folder, target_size=(200, 200), batch_size=186, class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(validation_folder, target_size=(200, 200), batch_size=184, class_mode='categorical')

Found 858 images belonging to 2 classes.
Found 186 images belonging to 2 classes.
Found 184 images belonging to 2 classes.


In [56]:
# Train the model
model.fit(train_generator, steps_per_epoch=39, epochs=10, validation_data=validation_generator, validation_steps=1)

Epoch 1/10
39/39 [==============================] - 284s 7s/step - loss: 53.8817 - accuracy: 0.4790 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
39/39 [==============================] - 290s 7s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/10
39/39 [==============================] - 289s 7s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/10
39/39 [==============================] - 290s 7s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/10
 8/39 [=====>........................] - ETA: 3:35 - loss: 0.6937 - accuracy: 0.4205